In [1]:
import json 

In [2]:
import numpy as np

In [3]:
import tensorflow as tf

C:\Users\hp\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\hp\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\hp\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\hp\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1)

In [4]:
from tensorflow import keras

In [5]:
from tensorflow.keras.models import Sequential

In [6]:
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D

In [7]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [8]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [9]:
from sklearn.preprocessing import LabelEncoder

In [10]:
with open('intents.json') as file:
    data = json.load(file)

In [11]:
training_sentences = []
training_labels = []
labels = []
responses = []

In [12]:
for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])

    if intent['tag'] not in labels:
        labels.append(intent['tag'])

num_classes = len(labels)

In [13]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

In [14]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [15]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 16)            16000     
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 16)                272       
_________________________________________________________________
dense_1 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 102       
Total params: 16,6

In [16]:
epochs = 500
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

Epoch 1/500
28/28 [==============================] - 2s 57ms/sample - loss: 1.7900 - acc: 0.2500
Epoch 2/500
28/28 [==============================] - 0s 0s/sample - loss: 1.7884 - acc: 0.2500
Epoch 3/500
28/28 [==============================] - 0s 107us/sample - loss: 1.7865 - acc: 0.2500
Epoch 4/500
28/28 [==============================] - 0s 321us/sample - loss: 1.7847 - acc: 0.2500
Epoch 5/500
28/28 [==============================] - 0s 143us/sample - loss: 1.7829 - acc: 0.2500
Epoch 6/500
28/28 [==============================] - 0s 214us/sample - loss: 1.7812 - acc: 0.2500
Epoch 7/500
28/28 [==============================] - 0s 179us/sample - loss: 1.7794 - acc: 0.2500
Epoch 8/500
28/28 [==============================] - 0s 679us/sample - loss: 1.7777 - acc: 0.2500
Epoch 9/500
28/28 [==============================] - 0s 107us/sample - loss: 1.7760 - acc: 0.2500
Epoch 10/500
28/28 [==============================] - 0s 178us/sample - loss: 1.7744 - acc: 0.2500
Epoch 11/500
28/28 [===

In [17]:
# to save the trained model
model.save("chat_model")

import pickle

# to save the fitted tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# to save the fitted label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)